# Data ETL pipeline

In [1]:
import sys
sys.path.append("../")
from src.service.provider import ProviderService
from src.prepare.data_etl import DataETL
import pandas as pd
from src.rag.types import RAGCategories

provider = ProviderService()

etl = DataETL(provider=provider, rag_config=provider.categories.sample)
df = pd.read_csv("../data/test_sample/docs.csv")
df.shape

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (44, 5) to 44
From (11, 4) to 11
From (8, 4) to 8


(2, 5)

In [2]:
etl.try_split_text(data_folder="test_sample", size=460, overlap=20)

From (2, 5) to Counter({2: 1, 1: 1})


[Document(page_content='Tiền thân của Trường Đại học Tôn Đức Thắng (Ton Duc Thang University: TDTU) là Trường Đại học Công nghệ Dân lập Tôn Đức Thắng, thành lập theo Quyết định 787/TTg-QĐ ngày 24/9/1997 của Thủ tướng Chính phủ. Trường do Liên đoàn lao động Thành phố Hồ Chí Minh sáng lập và quản lý thông qua Hội đồng quản trị do Chủ tịch Liên đoàn lao động Thành phố đương nhiệm làm chủ tịch.\n\nMục tiêu thành lập TDTU trong giai đoạn đầu là: thực hiện Chương trình 17/TU và Chỉ thị 13 của Thành ủy Thành phố Hồ Chí Minh về đào tạo, đào tạo lại, bồi dưỡng và nâng cao trình độ chuyên môn, tay nghề cho giai cấp công nhân Thành phố; phát triển nguồn nhân lực cho nhu cầu công nghiệp hoá - hiện đại hoá; góp phần đào tạo nhân tài, nhân lực, thực hiện nghiên cứu để phục vụ hệ thống sản xuất, xã hội ở Thành phố Hồ Chí Minh và các tỉnh phía Nam.\n\nNgày 28/01/2003, Thủ tướng Chính phủ ra Quyết định số 18/2003/TTg-QĐ chuyển đổi pháp nhân và đổi tên Trường Đại học Công nghệ Dân lập Tôn Đức Thắng thàn

## Upload docs

In [3]:
a = etl.upload_df_docs(
    rag_config=provider.categories.sample,
    data_folder="test_sample",
    size=460,
    overlap=20)

-> DONE: upload to elastic search 2 & 2 docs
-> DONE: upload to firebase at sample
-> DONE: upload to elastic search 1 & 1 docs
-> DONE: upload to firebase at sample
DONE


In [4]:
len(a)

2

In [5]:
etl.delete_doc(a[0])

In [6]:
etl.delete_doc(a[1])

## Upload 1 doc

In [2]:
category = "test_qa"

etl.upload_doc(
    db_category=category,
    data=df.iloc[1],
    text_splitter=etl.create_splitter(), 
    es=provider.get_elasticsearch_store(index=category))

-> DONE: upload to elastic search 1 docs
-> DONE: upload to firebase at test_qa


'- Bye:  Bye is a common phr...\n'

In [4]:
es = provider.get_elasticsearch_store(index=category)
a = es.similarity_search(query="bye")
a

[Document(page_content='Bye is a common phrase for departing', metadata={'id': '8af20725-b2e8-4f04-b5ce-0567de3a64ca'})]

In [7]:
id = a[0].metadata['id']
a = etl.fire.read_document(collection=category, document_id=id)
a

In [9]:
a.content

' Bye is a common phrase for departing'